
# Demonstration View

1. Compute Keep Steps
    1. Trajectory Analysis
    2. Compute Keep Steps
    3. Verify
2. Compute Masks
    1. Compute Mask
    2. Verify



View a demonstration by sliding through the frames and generate foreground segmentation.

This script creates the files that flowcontrol requires: `episode_0_keep.npz`, `episode_0.json`, etc.

# 0. Setup

In [ ]:
import os
import numpy as np
from tqdm import tqdm

def is_notebook():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True  # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False  # Probably standard Python interpreter

interactive = is_notebook()  # becomes overwritten
if interactive:
    get_ipython().run_line_magic('matplotlib', 'notebook')
    from ipywidgets import widgets, interact, Layout
    import matplotlib.pyplot as plt

In [ ]:
import copy
import json
from glob import glob

from robot_io.recorder.simple_recorder import load_rec_list, unprocess_seg

if interactive:    
    # set parameters here
    #recording, episode_num = "/home/argusm/lang/flowcontrol/flow_control/tmp_test/pick_n_place/", 0
    recording, episode_num = "/home/argusm/lang/flowcontrol/flow_control/tests/tmp_test/shape_sorting_rN/", 0
    #recording, episode_num = "/home/argusm/CLUSTER/robot_recordings/flow/sick_vacuum/17-19-19/", 0
else:
    # expect commandline input
    import sys
    if len(sys.argv) != 3:
        print("Usage: Demonstration_Viewer.py <episode_dir> <episode_num>")
    recording = sys.argv[1]
    episode_num = int(sys.argv[2])


if not os.path.isdir(recording):
    ValueError(f"Recording directory not found: {recording}")

segment_conf_fn = os.path.join(recording, "segment_conf.json")
keep_fn = os.path.join(recording, f"servo_keep.json")
mask_fn = os.path.join(recording, f"servo_mask.npz")

try:
    with open(segment_conf_fn, "r") as f_obj:
        orig_conf = json.load(f_obj)
        conf = copy.deepcopy(orig_conf)
except FileNotFoundError:
    orig_conf = None
    conf = [ (dict(name="color", color=(0, 0, 1), threshold=.72), dict(name="center")),
             (dict(name="color", color=(1, 0, 0), threshold=.90), dict(name="center")),
             (dict(name="color", color=(1, 0, 0), threshold=.90), dict(name="center"))]
    with open(segment_conf_fn, "w") as f_obj:
        json.dump(conf, f_obj)

# XXX
conf = dict(objects=dict(blue_block=[{'name': 'color', 'color': [0, 0, 1], 'threshold': 0.65}, {'name': 'center'}],
                         red_nest=[{'name': 'color', 'color': [1, 0, 0], 'threshold': 0.9}, {'name': 'center'}]),
            sequence=("blue_block","red_nest","red_nest"))
orig_conf = copy.deepcopy(conf)

rec = load_rec_list(recording)
actions = np.array([renv.get_action()["motion"] for renv in rec],dtype=object)
tcp_pos = np.array([renv.robot.get_tcp_pos() for renv in rec])
tcp_orn = np.array([renv.robot.get_tcp_orn() for renv in rec])
gripper_width = np.array([renv.robot.gripper.width() for renv in rec])
video_recording = np.array([renv.cam.get_image()[0] for renv in rec])

if "seg_mask" in rec[0].data["info"].item():
    masks_sim_l = [unprocess_seg(rec_el.data["info"].item()["seg_mask"])[0] for rec_el in rec]
    masks_sim = np.array(masks_sim_l)
    print("loaded segmentations.")
else:
    masks_sim = None

assert gripper_width.ndim == 1
num_frames = video_recording.shape[0]
max_frame = num_frames-1
print("loaded.")

# 1. Compute Keep Steps 

Decide which frames to keep, saved as per-frame boolean array.
Various options are possible, current choice is:
1. TCP Stationary Filter: find frames where movement is minimal.
2. Gripper Stable Filter: look at gripper motion and keep only those where gripper is stable.

In [ ]:
GRIPPER_OPEN, GRIPPER_CLOSE = 1.0, -1.0  # assume normalized actions

# use actions here instead of state position recordings as these
# are more direct and reliable
gr_actions = actions[:, -1]
keysteps = np.where(np.diff(gr_actions))[0].tolist()
keystep_names = ["" for _ in range(len(keysteps))]

# divide sequence into steps, defined by gripper action
segment_steps = np.zeros(num_frames)
segment_steps[np.array(keysteps)+1] = 1
segment_steps = np.cumsum(segment_steps).astype(int)

demonstration_type = ["navigate", "grasp", "grasp_insert"][len(keysteps)]

def check_gripper_opens(idx):
    # check that we transition open->close & iter segment
    assert gr_actions[idx] == GRIPPER_OPEN
    assert gr_actions[idx+1] == GRIPPER_CLOSE
    assert segment_steps[idx+1] == segment_steps[idx] + 1  # next

def check_gripper_closes(idx):
    # check that we transition open->close & iter segment
    assert gr_actions[idx] == GRIPPER_CLOSE
    assert gr_actions[idx+1] == GRIPPER_OPEN
    assert segment_steps[idx+1] == segment_steps[idx] + 1  # next

# run some checks
if demonstration_type == "grasp":
    assert(len(keysteps) == 1)
    check_gripper_opens(keysteps[0])
    keystep_names[0] = "gripper_open"    
elif demonstration_type == "grasp_insert":
    assert(len(keysteps) == 2)
    check_gripper_opens(keysteps[0])
    check_gripper_closes(keysteps[1])
    keystep_names[0] = "gripper_open"
    keystep_names[1] = "gripper_close"
else:
    raise NotImplementedError

print("demonstration type:", demonstration_type)
for kn, kidx in zip(keystep_names, keysteps):
    print(kn, "@", kidx)

### 1.1 TCP Stationary Filter

Slow robot motion indicates motion to a stable position, which we want to follow.

In [ ]:
vel_cont_threshold = .02  # [m/iter] if mean vel above this assume
vel_stable_threshold = .002  # [m/iter] if vel below this assume stable

def is_demo_continous(pos_vec):
    # check if the demonstration is conintous video or individual frames
    vel_vec = np.diff(pos_vec, axis=0)
    vel_scl = np.linalg.norm(vel_vec, axis=1)
    
    if np.mean(vel_scl) > vel_cont_threshold:
        print("Auto-detected: non-continous trajectory")
        return False
    
    return True

def get_stable_points(pos_vec):
    vel_vec = np.diff(pos_vec, axis=0)
    vel_scl = np.linalg.norm(vel_vec, axis=1)

    # This first loop gets minimal regions
    active = False
    start, stop = -1, -1
    min_regions = []
    for i in range(len(vel_scl)):
        if vel_scl[i] < vel_stable_threshold:
            if active:
               stop = i
            else:
                active = True
                start, stop = i, i
        else:
            if active:
                min_regions.append((start, stop))
                active = False
                start, stop = -1, -1

    # This second loop gets minimal value
    vel_stable = []
    for start, stop in min_regions:
        try:
            min_idx = start + 1 + np.argmin(vel_scl[start:stop])
        except ValueError:
            min_idx = 0
        if len(vel_stable) == 0 or vel_stable[-1] != min_idx:
            vel_stable.append(min_idx)
    return vel_stable, vel_scl

is_continous = is_demo_continous(tcp_pos)
if is_continous:
    vel_stable, vel_scl = get_stable_points(tcp_pos)
else:
    vel_vec = np.diff(tcp_pos, axis=0)
    vel_scl = np.linalg.norm(vel_vec, axis=1)
    vel_stable = list(range(len(tcp_pos)))
print("vel_stable", vel_stable)

### 1.2 Gripper Stationary Filter

Gripper motion makes servoing difficult, filter out those frames where it moves.

In [ ]:
def get_gripper_transitions(gripper_pos, diff_t=.0005, min_duration=5):
    # check that the gripper velocity is below diff_t for at least min_duration
    gripper_abs_vel = np.abs(np.diff(gripper_pos))
    stable = np.concatenate(([True,], gripper_abs_vel < diff_t))
       
    grip_stable = []
    grip_ends = []
    for i in range(len(stable)):
        snext = np.all(stable[i:min(i+min_duration, len(stable))])
        grip_stable.append(snext)
        if grip_stable[-2:] == [0, 1]:
            grip_ends.append(i-1)
            
    # fix edge case, gripper dosen't stop in demo
    if len(grip_ends) < len(keysteps):
        grip_ends.append(max_frame)

    grip_unstable = list(zip(keysteps, grip_ends))
    return grip_unstable

if is_continous:
    grip_unstable_intervals = get_gripper_transitions(gripper_width)    
else:
    grip_unstable_intervals = get_gripper_transitions(gripper_width, min_duration=1)    
print("grip_unstable", grip_unstable_intervals)

In [ ]:
# Option 1: trust grip unstable, filter out vel_stable
def filter_grip_unstable(vel_stable, grip_unstable_intervals):
    grip_stable_arr = np.ones(num_frames)
    for start, stop in grip_unstable_intervals:
        grip_stable_arr[start+1:stop+1] = False

    vel_stable_filtered = []
    for index in vel_stable:
        if grip_stable_arr[index]:
            vel_stable_filtered.append(index)
    return vel_stable_filtered, grip_stable_arr

# Option 2: trust vel_stable, override grip_stable
# this is probably a bit more reasonable.

vel_stable, grip_stable_arr = filter_grip_unstable(vel_stable, grip_unstable_intervals)
print("vel_stable", vel_stable, "(after filter with grip unstable)")

In [ ]:
def get_keep_dict_sparse(keysteps, vel_stable, max_frame):
    keep_dict = {}
    keep_dict[0] = dict(name="demo_start")
    
    # a) first update all stable
    for vel_s in vel_stable:
        keep_dict[int(vel_s)] = dict(name="vel_stable")
        
    # b) then ovewrite with grips (order is important here)
    for k_idx, k_name in zip(keysteps, keystep_names):
        keep_dict[k_idx] = dict(name=k_name)
        
    keep_dict[int(max_frame)] = dict(name="demo_end")
    keep_dict = {k: keep_dict[k] for k in sorted(keep_dict)}
    return keep_dict

keep_dict = get_keep_dict_sparse(keysteps, vel_stable, max_frame)    
print("keep_dict", list(keep_dict.keys()))

In [ ]:
from scipy.spatial.transform import Rotation as R

stationary_dist_threshold = 0.020  # [m/iter]

def get_rel_motion(start_pos, start_orn, finish_pos, finish_orn):
    # position
    pos_diff = finish_pos - start_pos
    ord_diff = R.from_quat(finish_orn).inv() * R.from_quat(start_orn)
    #assert ord_diff.magnitude() < .35, ord_diff.magnitude() # for now
    return pos_diff.tolist() + ord_diff.as_quat().tolist()

def filter_stationary(keep_dict, stationary_dist_threshold):
    remove_keys = []
    prior_key = None
    for key in keep_dict:
        if prior_key is None:
            prior_key = key
            continue
        rel_motion = get_rel_motion(tcp_pos[prior_key], tcp_orn[prior_key],
                                    tcp_pos[key], tcp_orn[key])
        rel_dist = np.linalg.norm(rel_motion[0:3])
        same_step = segment_steps[prior_key] == segment_steps[key]
        if rel_dist < stationary_dist_threshold and same_step:
            print("{} -> {}: {:.4f}".format(prior_key, key, float(rel_dist)),"(removing)")
            remove_keys.append(prior_key)
        else:
            prior_key = key
            
    keep_keys = keep_dict.keys() - remove_keys
    keep_dict_filtered = { key: keep_dict[key] for key in keep_keys}
    return keep_dict_filtered
        
keep_dict = filter_stationary(keep_dict, stationary_dist_threshold)
print("keep_dict", list(keep_dict.keys()), "(after filter stationary)")

In [ ]:
def set_grip_dist(keep_dict, max_dist=10):
    step_since_grasp = max_dist
    # Iterate backward and save dist to grasp
    for key in reversed(sorted(keep_dict)):
        name = keep_dict[key]["name"]
        if name.startswith("gripper_"):
            step_since_grasp = 0
        else:
            step_since_grasp = min(step_since_grasp+1, max_dist)
        keep_dict[key]["grip_dist"] = step_since_grasp

    prior_key = None
    for key in sorted(keep_dict):
        if prior_key is None:
            prior_key = key
            continue
        pre_dict = {}

        same_segment = segment_steps[key] == segment_steps[prior_key]
        if not same_segment:
            pre_dict["grip"] = gr_actions[key]

        if keep_dict[prior_key]["grip_dist"] < 2:
            rel_motion = get_rel_motion(tcp_pos[prior_key], tcp_orn[prior_key],
                                        tcp_pos[key], tcp_orn[key])
            pre_dict["rel"] = rel_motion
        else:
            abs_motion = [*tcp_pos[key], *tcp_orn[key]]
            pre_dict["abs"] = abs_motion

        keep_dict[key]["pre"] = pre_dict
        prior_key = key

        # double check that we retain all keep steps
        #assert(np.all([k in keep_dict.keys() for k in keysteps]))

set_grip_dist(keep_dict)

with open(keep_fn, 'w') as outfile:
    json.dump(keep_dict, outfile)
print("Saved to", keep_fn)

## 1. C. Verify keep frames

In [ ]:
if interactive:
    keep_array = np.zeros(segment_steps.shape)
    keep_array[sorted(keep_dict.keys())] = True
    fig, (ax, ax2) = plt.subplots(2, 1)
    line = ax.imshow(video_recording[0])
    ax.set_axis_off()
    ax2.plot(gripper_width*10, label="grip raw")
    ax2.plot(segment_steps/10, label="steps")
    ax2.plot(keep_array, label="keep")
    ax2.plot((gr_actions+1)/2, label="gripper action")
    ax2.set_ylabel("value")
    ax2.set_xlabel("frame number")
    vline = ax2.axvline(x=2, color="k")
    ax2.legend()

    def update(w):
        vline.set_data([w, w], [0, 1])
        line.set_data(video_recording[w])
        fig.canvas.draw_idle()
        if w in keep_dict:
            print(keep_dict[w])
            print()
    slider_w = widgets.IntSlider(min=0, max=max_frame, step=1, value=0,
                                 layout=Layout(width='70%'))
    interact(update, w=slider_w)

    print("What I want to know: do I servo y/n, do I translate?")
    # Convert this keep_array stuff into a dict
    # then do one iteration of look ahead to set a servoing flag.

### Show Gripper Motion
Show when the gripping is done, depending on gripper motion.

In [ ]:
# Plot gripper
if interactive:
    val, label = gripper_width, "gripper_pos"
    fig, (ax, ax2) = plt.subplots(2, 1)
    line = ax.imshow(video_recording[0])
    ax.set_axis_off()
    line1 = ax2.plot((gr_actions+1)/2, label="gripper action", color="r")
    line2 = ax2.plot(grip_stable_arr, label="grip stable")
    ax2.set_ylabel("value")
    ax2.set_xlabel("frame number")
    ax2r = ax2.twinx()
    line3 = ax2r.plot(val, label=label, color="b")
    vline = ax2.axvline(x=2, color="k")
    lns = line1+line2+line3
    labs = [l.get_label() for l in lns]
    ax2.legend(lns, labs)

    def update(w):
        print("{} @ {} is {}".format(label, w, val[w]))
        vline.set_data([w, w], [0, 1])
        line.set_data(video_recording[w])
        fig.canvas.draw_idle()

    slider_w = widgets.IntSlider(min=0, max=max_frame, step=1, value=0,
                                 layout=Layout(width='70%'))
    interact(update, w=slider_w)
    

### Show Velocities
Look at the end effector motion.

In [ ]:
if interactive:
    val, label =  vel_scl, "velocity"
    fig, (ax, ax2) = plt.subplots(2, 1)
    line = ax.imshow(video_recording[0])
    ax.set_axis_off()
    line1 = ax2.plot(tcp_pos[:,0], label="x")
    line2 = ax2.plot(tcp_pos[:,1], label="y")
    line3 = ax2.plot(tcp_pos[:,2], label="z")
    ax2.set_ylabel("position (x,y,z)")
    ax2.set_xlabel("frame number")
    ax2r = ax2.twinx()
    ax2r.set_ylabel("velocity/threshold")
    line4 = ax2r.plot(val, label=label, color="b")
    ax2r.axhline(y=vel_stable_threshold, linestyle="--", color="k")
    vline = ax2.axvline(x=2, color="k")
    lns = line1+line2+line3+line4
    labs = [l.get_label() for l in lns]
    ax2.legend(lns, labs)

    #ax2.legend()

    def update(w):
        print("{} @ {} is {}".format(label, w, val[w] if w<max_frame else "?"))
        vline.set_data([w, w], [0, 1])
        line.set_data(video_recording[w])
        fig.canvas.draw_idle()

    slider_w = widgets.IntSlider(min=0, max=max_frame, step=1, value=0,
                                 layout=Layout(width='70%'))
    interact(update, w=slider_w)

# 2. Compute Mask from Color Images

Mask out the foreground object so that foreground specific flow can be calculated.

In [ ]:
from skimage import measure
from scipy import ndimage
from demo_segment_util import mask_color, erode_mask, label_mask, mask_center

# create a segmentation mask
def get_mask(frame, step_conf, depth=None):
    """
    create segmentation mask for single frame
    Args:
        frame: input frame w x h x 3 [0,255] array
        i: index of frame, for indexing parameters
        threshold: threshold for color
        
    Returns:
        mask: binary numpy array, with True == keep
    """
    threshold = step_conf[0]["threshold"]    
    image = frame.copy()
    
    for seg_option in step_conf:
        name = seg_option["name"]
        
        if name == "color":
            color_choice = seg_option["color"]
            mask = mask_color(image, color_choice=color_choice, threshold=threshold)
            
        elif name == "erode":
            mask = erode_mask(mask)
            
        elif name == "height":
            raise NotImplementedError
            depth2 = transform_depth(depth, np.linalg.inv(T_tcp_cam))
            mask2 = get_mask_depth(depth2, 600, 1550)
            mask[mask2] = True
    
        elif name == "labels":
            raise NotImplementedError
            mask = ndimage.morphology.binary_closing(mask, iterations=4)
            mask = label_mask(mask, i)
    
        elif name == "imgheight":
            height_val = seg_option["height"]
            mask[:height_val, :] = False
            
        elif name == "center":
            mask = mask_center(mask)
            
    return mask

def get_cur_mask(i):
    # mask according to current fg object
    cur_step = segment_steps[i]
    cur_obj = conf["sequence"][cur_step]
    step_conf = conf["objects"][cur_obj]
    mask = get_mask(video_recording[i], step_conf)
    return mask

# Plot
if interactive:
    print("Colored stuff is keept (mask==True)")
    print("keysteps:", keysteps)
    print("segments: ", len(conf))

    fig, ax = plt.subplots(1, 1)
    line = ax.imshow(video_recording[0])
    ax.set_axis_off()
    prev_step = 0
    def update(i, t):
        cur_step = segment_steps[i]
        cur_obj = conf["sequence"][cur_step]
        global prev_step
        if cur_step != prev_step:
            # don't change order here, without double checking
            saved_t = conf["objects"][cur_obj][0]["threshold"]
            print(f"switching step {prev_step} -> {cur_step}, loading t={saved_t}")
            prev_step = cur_step
            slider_t.value = saved_t*100
        else:
            conf["objects"][cur_obj][0]["threshold"] = t/100
            
        mask = get_cur_mask(i)
        image = video_recording[i].copy()
        image[np.logical_not(mask)] = 255, 255, 255
        line.set_data(image)
        fig.canvas.draw_idle()

    slider_i = widgets.IntSlider(min=0, max=max_frame, step=1, value=0,
                                 layout=Layout(width='70%'))
    first_obj = conf["sequence"][0]
    slider_t = widgets.IntSlider(min=0, max=100, step=1, value=conf["objects"][first_obj][0]["threshold"]*100,
                                 layout=Layout(width='70%'))
    interact(update, i=slider_i, t=slider_t)

In [ ]:
masks = np.zeros(video_recording.shape[:3], dtype=bool)
switch_frame = keysteps
print("switching at:", switch_frame)
if orig_conf is None:
    orig_conf = conf

"""
# display changes files to selected.
for seg_option, seg_option_orig in zip(conf, orig_conf):
    c = seg_option[0]["color"]
    t = seg_option[0]["threshold"]
    t_i = seg_option_orig[0]["threshold"]
    if t != t_i:
        print("c={}, t={} | t'={}".format(c, t, t_i))
    else:
        print("c={}, t={}".format(c, t))
"""
obj_ids = {}
obj_ids_list = []
for i, obj in enumerate(conf["objects"]):
    obj_ids[obj] = i+1
    obj_ids_list.append(i+1)
    print(f"{obj} -> {i+1}")


fg_obj = []
masks_list = []
for i in tqdm(range(len(video_recording))):
    # get foreground object
    cur_step = segment_steps[i]
    cur_obj = conf["sequence"][cur_step]
    fg_obj.append(obj_ids[cur_obj])
    
    m_masks = []
    for obj_name in conf["objects"]:
        mask = get_mask(video_recording[i], conf["objects"][obj_name])
        m_masks.append(mask)
    
    overlapp = np.sum(m_masks, axis=0) > 1
    if np.any(overlapp):
        print("WARNING: There is overlapp.")
    masks_list.append(m_masks)
    
fg_obj = np.array(fg_obj)
assert fg_obj.ndim == 1

masks_list = np.array(masks_list)
masks_list = masks_list.transpose(1, 0, 2, 3).astype(np.uint8)
obj_ids_arr = np.array(obj_ids_list).reshape(-1, 1, 1, 1)
masks_list = obj_ids_arr*masks_list
masks_list = masks_list.sum(axis=0)
assert masks_list.ndim == 3


print(round(np.mean(masks) * 100), "% of pixels fg")
np.savez_compressed(mask_fn, mask=masks, fg=fg_obj)
print("Saved to", mask_fn)

if conf != orig_conf:
    print("Warning using new conf values")

## 2. B. Verify Masking Results

In [ ]:
if interactive:
    fig, ax = plt.subplots(1)
    handle = ax.imshow(masks[0])
    ax.set_axis_off()
    
    def update(i):
        image = video_recording[i].copy()
        mask = masks_list[i] == fg_obj[i]
        print(round(np.mean(mask)*100), "% fg, mask shape", mask.shape)
        image[np.logical_not(mask)] = 255, 255, 255
        handle.set_data(image)
        fig.canvas.draw_idle()

    slider_i2 = widgets.IntSlider(min=0, max=max_frame, step=1, value=0,
                                 layout=Layout(width='70%'))
    interact(update, i=slider_i2)

In [ ]:
if masks_sim is not None:
    for i in range(num_frames):
        image = video_recording[i].copy()
        mask = masks_list[i] == fg_obj[i]
        # mask segmentation mask(gt) with fg mask (computed)
        ma = np.ma.array(masks_sim[i], mask=np.logical_not(mask))
        ma_unique = np.unique(ma, return_counts=True)
        # unique is sorted by size, pick the biggest
        idx_largest = np.where(ma_unique[0])[-1][0]
        seg_id, mask_count = ma_unique[0][idx_largest], ma_unique[1][idx_largest]
        seg_count = np.sum(masks_sim[i] == seg_id)
        # test how much we segmented / how much there is
        score = mask_count / seg_count
        assert score > .9
        
    print("Segmentation test passed.")

# 3.  Mask from Simulation

This cell extracts foreground masks from simulation recordings. It does this by looking at the recordings info variables, where a anchor object UID can be specified. This is usually done by the task policy.

The move_anchor is the object with which we are moving relative to, this is most often but not always the object of interest or the foreground object.

In [ ]:
if masks_sim is not None:    
    fg_obj_sim = np.array([rec_el.data["info"].item()["move_anchor"] for rec_el in rec])    
    #print(foreground_uids)
    #print(np.unique(masks_sim))
    
    np.savez_compressed(mask_fn, mask=masks_sim, fg=fg_obj_sim)
    print("Saved to", mask_fn)


In [ ]:
if interactive:
    fig, ax = plt.subplots(1)
    handle = ax.imshow(video_recording[0])
    ax.set_axis_off()
    
    def update(i):
        image = video_recording[i].copy()
        mask = masks_sim[i] == fg_obj_sim[i]
        print(round(np.mean(mask)*100), "% fg, mask shape", mask.shape)
        image[np.logical_not(mask)] = 255, 255, 255
        handle.set_data(image)
        fig.canvas.draw_idle()

    slider_i2 = widgets.IntSlider(min=0, max=max_frame, step=1, value=0,
                                 layout=Layout(width='70%'))
    interact(update, i=slider_i2)

# 4. Masking based on Depth